In [1]:
import time
from selenium import webdriver
import pandas as pd

In [2]:
import chromedriver_autoinstaller

driver_name = chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path
time.sleep(1)

In [3]:
driver = webdriver.Chrome(driver_name)

In [4]:
url = 'http://www.bursamalaysia.com/market/securities/equities/prices/#/?filter=BS02'
page = driver.get(url)
time.sleep(2)

In [5]:
df = pd.read_html(driver.page_source)[0]
df = pd.read_html(io.StringIO(str(driver.page_source)))[0]
df.head()

,No,Name,Code,REM,Last Done,LACP,CHG,%CHG,Vol ('00),BUY Vol ('00),BUY,SELL,SELL Vol ('00),HIGH,LOW,stock_id


In [6]:
df = df.append(pd.read_html(driver.page_source)[1])
df.head()

,No,Name,Code,REM,Last Done,LACP,CHG,%CHG,Vol ('00),BUY Vol ('00),BUY,SELL,SELL Vol ('00),HIGH,LOW,stock_id
0,1,BCMALL,0187,NaN,0.025,0.025,-,-,816110,528367,0.020,0.025,248,0.025,0.025,0187
1,2,HSI-CQW,0651QW,NaN,0.245,0.265,-0.020,-7.55,622096,3920,0.245,0.250,6352,0.305,0.235,0651QW
2,3,LGMS [S],0249,NaN,0.820,0.770,+0.050,+6.49,555397,1958,0.820,0.825,3204,0.840,0.795,0249
3,4,YONGTAI [S],7066,NaN,0.090,0.085,+0.005,+5.88,551707,14091,0.090,0.095,53780,0.095,0.085,7066
4,5,MNC [S],0103,NaN,0.015,0.015,-,-,526094,4566,0.015,0.020,844136,0.020,0.010,0103


### selenium에서 BeautifulSoup 사용하기
- soup = bs(driver.page_source, 'html.parser')

In [7]:
from bs4 import BeautifulSoup as bs

soup = bs(driver.page_source, 'html.parser')
img_source = soup.select('#logo') # return list

In [8]:
img_source[0]

<a class="d-lg-inline-block" href="/" id="logo">
<img alt="BURSA MALAYSIA" class="logo" src="/sites/5bb54be15f36ca0af339077a/theme/images/logo.png?1655349846"/>
</a>

In [9]:
img_source[0].img['alt'] # img tag의 alt attribute

'BURSA MALAYSIA'

### selenium WebElement
- WebElement 다루기
- element.get_attribute('innerHTML'): 지정한 요소 태그를 제외한 내부 태그만 값을 가져옴. string(str) not DOM element
- element.get_attribute('outerHTML'): 지정한 요소 태그를 포함하여 포함하여 내부전체 값을 가져옴. string(str) not DOM element

In [10]:
logo = driver.find_element_by_id('logo')

In [11]:
logo

<selenium.webdriver.remote.webelement.WebElement (session="3a669190ffaeec0ad4942050f07e2892", element="a063be54-3cd5-42be-a9bb-6b8bff38a858")>

In [12]:
logo.get_attribute('innerHTML') # 지정한 요소 태그(id='logo')를 제외한 내부 태그만 값을 가져옴. string not DOM element

'\n<img src="/sites/5bb54be15f36ca0af339077a/theme/images/logo.png?1655349846" class="logo" alt="BURSA MALAYSIA">\n'

In [13]:
logo.get_attribute('outerHTML')# 지정한 요소 태그(id='logo')를 포함하여 내부전체 값을 가져옴. string not DOM element

'<a href="/" class="d-lg-inline-block" id="logo">\n<img src="/sites/5bb54be15f36ca0af339077a/theme/images/logo.png?1655349846" class="logo" alt="BURSA MALAYSIA">\n</a>'

In [14]:
logo.find_element_by_css_selector('a > img').get_attribute('src') # element 처리 명령어에 따라 진행

'https://www.bursamalaysia.com/sites/5bb54be15f36ca0af339077a/theme/images/logo.png?1655349846'

In [15]:
logo_soup = bs(logo.get_attribute('outerHTML'), 'html.parser') # string을 html형식으로 변환

In [16]:
logo_soup

<a class="d-lg-inline-block" href="/" id="logo">
<img alt="BURSA MALAYSIA" class="logo" src="/sites/5bb54be15f36ca0af339077a/theme/images/logo.png?1655349846"/>
</a>

In [17]:
logo_soup.a.img['alt']

'BURSA MALAYSIA'

In [18]:
logo_soup.a.img

<img alt="BURSA MALAYSIA" class="logo" src="/sites/5bb54be15f36ca0af339077a/theme/images/logo.png?1655349846"/>

In [19]:
type(logo_soup.a.img)

bs4.element.Tag

In [20]:
logo_soup.get_attribute("alt") # WebElement type이 아니어서 .get_attriute사용할 수 없음.

TypeError: 'NoneType' object is not callable

In [21]:
driver.close()
driver.quit()